In [3]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, \
    TrainingArguments
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
from pathlib import Path
from pynvml import *
import torch
import argparse
import sys
import random
import pandas as pd
import numpy as np
import itertools
import evaluate
import math

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used // 1024 ** 2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()


def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=128)


In [4]:
# parser = argparse.ArgumentParser(description='Task-adaptive model')
# parser.add_argument('--model', help='Path to BERT-like model')
# parser.add_argument('--model_name',
#                     help='Name of the BERT-like model. Default = "" which corresponds to ClinicalBERT',
#                     default='')
# config = parser.parse_args(sys.argv[1:])
task = 'sentiment' 
MODEL = 'UFNLP/gatortron-base'

# set seed
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


In [5]:
# Create task Dataset
task_dt_path = 'task_adapt_sentences.csv'
sentences = pd.read_csv(task_dt_path, header=None)
sentences = sentences[0].str.lower().str.split('.').values.tolist()
sentences = list(itertools.chain.from_iterable(sentences))

train, dev = train_test_split(sentences, test_size=0.2, random_state=42)

task_dt = {'train': {},
           'dev': {}}
           #'test': {}}

for s in train:
    task_dt['train'].setdefault('text', list()).append(s)

for s in dev:
    task_dt['dev'].setdefault('text', list()).append(s)

task_dt = DatasetDict({k: Dataset.from_dict(task_dt[k]) for k in task_dt.keys() if k != 'test'})
task_dt.flatten()

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.add_tokens(['[DATE]', '[TIME]'], special_tokens=True)
tkn_dt = task_dt.map(tokenize_function, batched=True, num_proc=4)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Fatal Python error: config_get_locale_encoding: failed to get the locale encoding: nl_langinfo(CODESET) failed
Python runtime state: preinitialized



Map (num_proc=4):   0%|          | 0/346 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/87 [00:00<?, ? examples/s]

In [6]:
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    model = AutoModelForMaskedLM.from_pretrained(
        MODEL,
        from_tf=False).to('cuda')
    print_gpu_utilization()
else:
    model = AutoModelForMaskedLM.from_pretrained(
        MODEL,
        from_tf=False)
model.resize_token_embeddings(len(tokenizer))

batch_size = 4 # [4, 8, 16]
logging_steps = len(task_dt["train"]) // batch_size

training_args = TrainingArguments(
    output_dir="runs/ta_pretraining",
    evaluation_strategy="epoch",
    learning_rate=5e-6, # [5e-6, 1e-5]
    num_train_epochs=5, 
    weight_decay=0.01, # [1e-6, 1e-4, 1e-8]
    warmup_ratio=0.01, #[1e-6, 1e-4, 1e-8]
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy='epoch',
    save_total_limit=2,
    logging_steps=logging_steps,
    adam_epsilon=1e-6, # try to add AdamW optimizer post changing the batch size
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tkn_dt['train'],
    eval_dataset=tkn_dt['dev'],
    data_collator=data_collator   
)

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


>>> Perplexity: 8.53


In [7]:
result = trainer.train()
print(result)

/Users/valena17/miniconda3/envs/torch-gpu/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.219600,1.611951
2,1.943800,1.812136
3,1.919200,1.327535
4,1.938300,1.818197
5,1.652100,1.596408


TrainOutput(global_step=435, training_loss=1.9247360437765888, metrics={'train_runtime': 3663.7745, 'train_samples_per_second': 0.472, 'train_steps_per_second': 0.119, 'total_flos': 186999608016000.0, 'train_loss': 1.9247360437765888, 'epoch': 5.0})


In [8]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 4.89
